In [1]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

In [2]:
# !pip3 install transformers==4.34.1

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# tokenizer = AutoTokenizer.from_pretrained('aisingapore/sealion7b', trust_remote_code = True)
# model = AutoModelForCausalLM.from_pretrained(
#     'aisingapore/sealion7b',
#     trust_remote_code = True,
#     torch_dtype = torch.float16,
#     device_map="cuda:0"
# )
tokenizer = AutoTokenizer.from_pretrained('aisingapore/sealion7b', trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained(
    'aisingapore/sealion7b',
    trust_remote_code = True,
    torch_dtype = torch.float16,
    device_map="auto"
)

/home/cybertron/anaconda3/envs/sealion/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.53s/it]


In [4]:
from tqdm import tqdm
import os
import random
torch.manual_seed(404)
random.seed(404)

In [5]:

data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/sejarah-spm/data/'
with open(data_folder+ 'sejarah_spm') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    #objektif = t.split('objektif: ')[1].split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)
    print(data)
    print()

{'no': '1', 'soalan': 'Apakah yang dimaksudkan dengan wilayah pengaruh?\nA. Kawasan tanah jajahan dan takluk raja\nB. Kawasan yang dinaungi oleh raja dan kerabatnya\nC. Kawasan yang rakyatnya menerima dan memperakui pemerintahan seseorang raja\nD. Kawasan yang rakyatnya menganggap raja sebagai tuhan atau wakil tuhan', 'jawapan': 'C'}

{'no': '2', 'soalan': 'Bagaimanakah Raja dalam kerajaan Alam Melayu memperkukuhkan kedudukannya?\nA. Ketaatan rakyat kepada raja\nB. Rakyat dianggap suci\nC. Rakyat menyembah raja\nD. Raja menjalankan kerahan tenaga', 'jawapan': 'A'}

{'no': '3', 'soalan': 'Apakah faktor perbezaan idea nasionalisme antara negara dengan zaman?\nA. Politik\nB. Darjat\nC. Keturunan\nD. Ekonomi', 'jawapan': 'C'}

{'no': '4', 'soalan': "Maklumat berikut berkaitan dengan pendapat tokoh mengenai nasionalisme. \nKita hendak menzahirkan suatu bangsa yang lebih tegap dan kuat, boleh\nberdiri di atas tapak kaki kita sendiri dan mengambil tempat kita yang\nsebenar di sisi bangsa-bang

In [6]:
arange = set(range(len(questions)))

In [7]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

In [8]:
i = 1
prompts = []

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts).strip()
print(prompt)

soalan: Bagaimanakah Raja dalam kerajaan Alam Melayu memperkukuhkan kedudukannya?
A. Ketaatan rakyat kepada raja
B. Rakyat dianggap suci
C. Rakyat menyembah raja
D. Raja menjalankan kerahan tenaga
jawapan:


In [9]:
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
inputs

{'input_ids': tensor([[128148, 249853,   9939,  42675,  15095,  58969,   3294,  39014,  43961,
          69503, 107003,    852,  13500, 104857,   1905, 249905,      4, 249843,
         249835,    532,  13619,   4770,  48510,   9606, 149315,      4, 249850,
         249835,  90172,  48501,  10666, 249819,      4, 249844, 249835,  90172,
          83663,    626, 149315,      4, 249852, 249835,  58969,  34964,   7448,
           5704,  51830,      4,  64068,   3329, 249853]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}

In [10]:
inputs.pop('token_type_ids', None)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], device='cuda:0')

In [11]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=3,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.05,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]).split('jawapan:')[1].strip().split())

['1.']


In [16]:
for i in tqdm(range(len(questions))):
    
    shots = random.sample(arange - {i}, 3)
    prompts = []
    for no, s in enumerate(shots):
        prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    # print(prompt)
    # break
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
    inputs.pop('token_type_ids', None)
    repeat = []
    for _ in range(5):
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=3,
                top_p=0.95,
                top_k=50,
                temperature=0.5,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            r = tokenizer.decode(r[0]).split('jawapan:')[-1].strip().split()
            repeat.append(r[0].replace('.', '').replace('</s>', '').split('\\')[0].split('/')[0])
    
        except Exception as e:
            print(e)
            pass
    
    questions[i]['output'] = repeat

  0%|                                                                                                                  | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_22018/2190211999.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  shots = random.sample(arange - {i}, 3)
  0%|                                                                                                                  | 0/20 [00:00<?, ?it/s]

Contoh soalan 1
soalan: Apakah faktor perbezaan idea nasionalisme antara negara dengan zaman?
A. Politik
B. Darjat
C. Keturunan
D. Ekonomi
jawapan: C

Contoh soalan 2
soalan: Jawatankuasa Kerja ditubuhkan pada 25 Julai 1946. Apakah cadangan yang disarankan oleh jawatankuasa tersebut?
A. Menyatukan sesetengah negeri Melayu dalam satu pentadbiran
B. Menjaga kepentingan orang Melayu sahaja
C. Mengenakan syarat kewarganegaraan yang ketat
D. Menubuhkan Dewan Rakyat dan Dewan Negara
jawapan: C

Contoh soalan 3
soalan: Dato’ Onn Jaafar telah dilantik sebagai Ahli Hal Ehwal Dalam Negeri Persekutuan Tanah Melayu. Apakah tanggungjawab beliau ?
A. Mendaftarkan kerakyatan
B. Menyelia perancangan bandar
C. Menyediakan kemudahan asas
D. Menyeragamkan sukatan pelajaran
jawapan: A

soalan: Apakah yang dimaksudkan dengan wilayah pengaruh?
A. Kawasan tanah jajahan dan takluk raja
B. Kawasan yang dinaungi oleh raja dan kerabatnya
C. Kawasan yang rakyatnya menerima dan memperakui pemerintahan seseorang ra

In [13]:
import json

with open('output-3shot-aisingapore-sealion7b.json', 'w') as fopen:
    json.dump(questions, fopen)

In [14]:
def most_common(l):
    return max(set(l), key=l.count)

In [15]:
filtered = [q for q in questions if 'output' in q]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

20.0